# CS 434: Implementation Assignment 1

## Linear Regression

1. Load the training data into the corresponding $X$ and $Y$ matrices, where $X$ stores the features and $Y$ stores the desired outputs. The rows of $X$ and $Y$ correspond to the examples and the columns of $X$ correspond to the features. Introduce the dummy variable to $X$ by adding an extra column of ones to $X$ (You can make this extra column to be the first column. Changing the position of the added column will only change the order of the learned weight and does not matter in practice).  Compute the optimal weight vector $w$ using $w = (X^T X)^{−1}X^T Y$. Feel free to use existing numerical packages (e.g., numpy) to perform the computation. Report the learned weight vector.

In [1]:
import numpy as np
from numpy.linalg import inv
import pandas as pd

In [2]:
df = pd.read_csv('housing_train.csv', header=None)
df.columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE',
              'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.15098,0.0,10.01,0,0.5470,6.021,82.6,2.7474,6,432,17.8,394.51,10.30,19.2
1,12.04800,0.0,18.10,0,0.6140,5.648,87.6,1.9512,24,666,20.2,291.55,14.10,20.8
2,0.34940,0.0,9.90,0,0.5440,5.972,76.7,3.1025,4,304,18.4,396.24,9.97,20.3
3,0.03578,20.0,3.33,0,0.4429,7.820,64.5,4.6947,5,216,14.9,387.31,3.76,45.4
4,1.65660,0.0,19.58,0,0.8710,6.122,97.3,1.6180,5,403,14.7,372.80,14.10,21.5


In [3]:
X = df.iloc[:, :-1].values
X = np.insert(X, 0, 1, axis=1)
y = df.iloc[:, -1].values

In [4]:
w = inv(X.T.dot(X)).dot(X.T.dot(y))
w

array([ 3.67103960e+01, -1.10220511e-01,  4.25270181e-02,  9.94268803e-03,
        4.03688262e+00, -1.81193844e+01,  3.91213593e+00, -3.24572263e-03,
       -1.61764599e+00,  3.51469633e-01, -1.35490385e-02, -8.88849879e-01,
        9.33221332e-03, -5.87431614e-01])

In [5]:
df_test = pd.read_csv('housing_test.csv', header=None)
X_test = df_test.iloc[:, :-1].values
X_test = np.insert(X_test, 0, 1, axis=1)
y_test = df_test.iloc[:, -1].values

In [6]:
# Training data ASE
pred_train = X.dot(w)
errors_train = y - pred_train
ASE_train = errors_train.T.dot(errors_train)/len(X)
ASE_train

21.635964362901735

In [7]:
# Testing data ASE
pred_test = X_test.dot(w)
errors_test = y_test - pred_test
ASE_test = errors_test.T.dot(errors_test)/len(X_test)
ASE_test

23.690360525687108

In [8]:
X = np.delete(X, 0, axis=1)
X_test = np.delete(X_test, 0, axis=1)

In [9]:
w = inv(X.T.dot(X)).dot(X.T.dot(y))
w

array([-0.09774079,  0.043473  , -0.0145373 ,  4.43876298, -2.93794405,
        5.98021602, -0.00944101, -1.07952617,  0.18834311, -0.0089222 ,
       -0.33255837,  0.01473948, -0.48346682])

In [10]:
# Training data ASE
pred_train = X.dot(w)
errors_train = y - pred_train
ASE_train = errors_train.T.dot(errors_train)/len(X)
ASE_train

23.96275049259436

In [11]:
# Testing data ASE
pred_test = X_test.dot(w)
errors_test = y_test - pred_test
ASE_test = errors_test.T.dot(errors_test)/len(X_test)
ASE_test

25.9387857057457

## Logistic Regression

In [ ]:
class LogisticRegression:
    def __init__(self, eta = 0.01, epoch = 15000):
        self.eta   = eta
        self.epoch = epoch

    def fit(self, X, y):
        self.weights = np.zeros(1 + X.shape[1])
        self.costs   = []

        for _ in range(self.epoch):
            output = self.sigmoid(self.net_input(X))
            errors = y - output
            self.weights[1:] += self.eta * X.T.dot(errors)
            self.weights[0]  += self.eta * errors.sum()
            cost = self.cost(X, y)
            self.costs.append(cost)
        return self

    @staticmethod
    def sigmoid(X):
        return 1 / (1 + np.exp(-X))

    def cost(self, X, y):
        net_input = self.net_input(X)
        net_input_pos = net_input[y == 1]
        net_input_neg = net_input[y == 0]
        cost = - (1 / X.shape[0]) \
                * (np.sum(np.log(self.sigmoid(net_input_pos) + 10**(-16))) 
                    + np.sum(np.log(1 - self.sigmoid(net_input_neg) + 10**(-16))))
        return cost

    def net_input(self, X):
        return np.dot(X, self.weights[1:]) + self.weights[0]

    def predict(self, X, threshold = 0.5):
        return self.sigmoid(self.net_input(X)) >= threshold